# Pré-Processamento dos Dados

## Título:

**Predição de preço de imóveis**

## Membros:

*   Adrisson Rogério Samersla
*   Nickolas Batista Mendonça Machado
*   Thayna Pires Baldão



# Setup

In [1]:
# Importando os pacotes necessários para a análise

import os

import pandas            as pd
# import geopandas         as gpd
import numpy             as np
import scipy             as sc
import matplotlib.pyplot as plt
import seaborn           as sns

from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import mapclassify

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.reset_option('max_colwidth')

In [2]:
# Baixando o dataset

dataset_dir = "../dataset"
has_dataset_dir = os.path.isdir(dataset_dir)
if (not has_dataset_dir):
  # Link of dataset folder: 
  # https://drive.google.com/file/d/1S4rBgtuogAGr_WIcF-FIPaULfGlB9MRs/view?usp=sharing
  gdd.download_file_from_google_drive(file_id='1S4rBgtuogAGr_WIcF-FIPaULfGlB9MRs',
                                      dest_path='../dataset.zip',
                                      showsize=True,
                                      unzip=True)

In [3]:
# Lendo a base de dados

df = pd.read_csv(dataset_dir + '/dataset.csv')
print("Formato dos dados: ", df.shape)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))

Formato dos dados:  (1000000, 24)
#Exemplos: 1000000
#Atributos: 24


In [4]:
df.dtypes

id                           int64
property_id                 object
created_on                  object
operation                   object
property_type               object
place_name                  object
place_with_parent_names     object
country_name                object
state_name                  object
geonames_id                float64
lat_lon                     object
lat                        float64
lon                        float64
currency                    object
surface_total_in_m2        float64
surface_covered_in_m2      float64
floor                      float64
rooms                      float64
expenses                   float64
description                 object
title                       object
image_thumbnail             object
collected_on                object
price                      float64
dtype: object

In [5]:
df.head()

,id,property_id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,lat,lon,currency,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,title,image_thumbnail,collected_on,price
0,9641098,9e065715d8b6c69bc930c185901aaf71e3be3d54,2017-04-28,sell,apartment,Campo Belo,|Brasil|São Paulo|São Paulo|Campo Belo|,Brasil,São Paulo,NaN,"-23.616325,-46.67662",-23.616325,-46.676620,BRL,NaN,80.0,NaN,NaN,1065.0,Atualizado em 05/06/2017. O projeto arquitetôn...,"Apartamento com 2 quartos e 2 Suites, São Paul...",https://thumbs4.properati.com/2/tQXG8R8vA-PsM4...,2017-08,900000.0
1,7965368,f6b640522dd15dd3667850718f52aa02516b04ff,2017-05-25,sell,apartment,Porto Alegre,|Brasil|Rio Grande do Sul|Porto Alegre|,Brasil,Rio Grande do Sul,NaN,"-30.05175,-51.182068",-30.051750,-51.182068,BRL,91.0,91.0,NaN,NaN,NaN,"APARTAMENTO com 3 dormitórios , sendo 1 suíte,...",Apartamento em Petrópolis,https://thumbs4.properati.com/6/gy-6GDbU5Ccx_I...,2017-07,750000.0
2,6779668,39a954f5d56137f6fe547af707626eafa6a3869c,2015-06-26,sell,apartment,Bessa,|Brasil|Paraíba|João Pessoa|Bessa|,Brasil,Paraíba,NaN,NaN,NaN,NaN,BRL,NaN,NaN,NaN,NaN,NaN,Edifcio: 10(dez) apartamentos; 01(uma) vaga ga...,Venda Apartamento Joo Pessoa/Paraba: 3 quartos...,https://thumbs4.properati.com/8/DL8fMA8z1oRVgv...,2015-12,235000.0
3,10187959,ae0282c0e20210cfaf030e57edf85b9d5d6c13f4,2016-02-22,sell,apartment,São Lourenço,|Brasil|Paraná|Curitiba|São Lourenço|,Brasil,Paraná,NaN,"-25.413214,-49.271294",-25.413214,-49.271294,BRL,NaN,168.0,NaN,NaN,NaN,Linda cobertura no bairro São Lourenço com uma...,"Linda Cobertura duplex - 168 m² - 2 suítes, 2 ...",https://thumbs4.properati.com/2/J6USW1fALKsXov...,2016-06,880000.0
4,1147630,f83c204dd1baf29c6460826ee829be00f751f676,2016-09-09,sell,house,Vila Formosa,|Brasil|São Paulo|São Paulo|Vila Formosa|,Brasil,São Paulo,NaN,"-23.56229,-46.521248",-23.562290,-46.521248,BRL,NaN,145.0,NaN,2.0,NaN,"Lindo sobrado, muito bem construído com ótimo ...",Sobrado novo para venda na Vila Formosa.,https://thumbs4.properati.com/0/bXqmIc-TKt1wua...,2017-03,580000.0


# Seleção Manual de Atributos


In [6]:
ignore_columns = [
    'id',
    'property_id',
    'operation',
    'place_name',
    'place_with_parent_names',
    'country_name',
    'state_name',
    'geonames_id',
    'lat_lon',
    'currency',
    'floor',
    'description',
    'title',
    'image_thumbnail',
    'collected_on'
]

def manual_selection(dataset):
    return dataset.drop(columns=ignore_columns)

#filtered = manual_selection(df)
# for col in ignore_columns:
#     assert col not in manual_selection(df).columns

# Limpeza de dados

## Eliminando Inconsistências

- Moedas Estrangeiras

In [7]:
df.currency.astype('str').value_counts()

BRL    999806
MXN       103
USD        87
COP         4
Name: currency, dtype: int64

In [8]:
def filter_currency(dataset):
    positions = dataset['currency'] == 'BRL'
    return dataset[positions]

#df = filter_currency(df).currency.astype('str').value_counts()

- Objetos Fora do Brasil

In [9]:
df.state_name.astype('str').value_counts()

São Paulo              704145
Rio Grande do Sul       90626
Rio de Janeiro          71693
Santa Catarina          27613
Minas Gerais            27401
Paraná                  18083
Ceará                   11472
Espírito Santo          11056
Paraíba                 10599
Bahia                    8578
Rio Grande do Norte      4981
Goiás                    2929
Pernambuco               2645
Distrito Federal         2300
Pará                     1346
Mato Grosso               899
Amazonas                  831
Tocantins                 714
Alagoas                   610
Maranhão                  568
Sergipe                   314
Mato Grosso do Sul        190
Rondônia                  139
Piauí                     123
Amapá                     123
Acre                       16
Outros países               4
Roraima                     2
Name: state_name, dtype: int64

In [10]:
def filter_latlon(dataset):
    data = dataset[['lat', 'lon']]

    # data = data.dropna()
    # points = gpd.points_from_xy(data.lon, data.lat, crs=map_df.crs)
    # points = gpd.GeoSeries(points)
    # positions_in = map_df.contains(points)

    positions = data.lat.notna().squeeze() & data.lon.notna().squeeze()

    lon_max = -30.0
    lon_min = -80.0
    lat_min = -40.0
    lat_max = 10.0
    return dataset.loc[
        positions & 
        (dataset.state_name != "Outros países") &
        ((data.lat > lat_min) & (data.lat < lat_max)) & 
        ((data.lon > lon_min) & (data.lon < lon_max))
    ]

#filter_latlon(df).state_name.astype('str').value_counts()

## Eliminando Outliers

In [11]:
def remove_outliers(dataset, cols):
    Q1 = dataset[cols].quantile(0.25)
    Q3 = dataset[cols].quantile(0.75)
    IQR = Q3 - Q1
    return dataset[~((dataset[cols] < (Q1 - 1.5 * IQR)) |(dataset[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

## Lidando com dados incompletos

- `expenses`

In [12]:
def fill_expenses(dataset):
    dataset['expenses'].fillna(0.0, inplace=True)
    return dataset

- `surface_total_in_m2` e `surface_covered_in_m2`

In [13]:
def fill_surface_total_with_surface_covered(dataset):
    dataset['surface_total_in_m2'].fillna(dataset['surface_covered_in_m2'], inplace=True)
    return dataset

def fill_surface_covered_with_surface_total(dataset):
    dataset['surface_covered_in_m2'].fillna(dataset['surface_total_in_m2'], inplace=True)
    return dataset

def fill_remaining_objects_without_surfaces(dataset):
    dataset['surface_total_in_m2'].fillna(dataset['surface_total_in_m2'].mean(), inplace=True)
    dataset['surface_covered_in_m2'].fillna(dataset['surface_covered_in_m2'].mean(), inplace=True)
    return dataset

def drop_remaining_objects_without_surfaces(dataset):
    dataset.dropna(subset=['surface_total_in_m2'], how='all', inplace=True)
    return dataset

- `rooms`

In [14]:
def fill_rooms(dataset):
    dataset['surface_category'] = pd.qcut(dataset['surface_covered_in_m2'],4)
    dataset['rooms'] = dataset['rooms'].fillna(dataset.groupby('surface_category')['rooms'].transform('mean').round())
    return dataset.drop(columns=['surface_category'])

# Transformando dados

## Conversão simbólico-numérico

- `property_type`

In [15]:
def convert_property_type(dataset):
    oe_style = OneHotEncoder(sparse=False)
    oe_results =  oe_style.fit_transform(dataset[['property_type']])
    pd_results = pd.DataFrame(oe_results, columns= oe_style.categories_)

    dataset.reset_index(drop=True, inplace=True)    # While filtering, pandas keeps the old indexes
    dataset = dataset.join(pd_results)              # which breaks the join

    dataset.rename(columns=lambda s : str(s).replace(",", "").replace("(", "").replace(")", "").replace("\'", ""), inplace=True)
    dataset = dataset.drop(columns=['property_type'])
    return dataset[[c for c in dataset if c not in ['price']] + ['price']]

- `created_on`

In [16]:
def convert_created_on(dataset):
    ord_enc = OrdinalEncoder()
    print(dataset.head(),dataset.tail())
    dataset['created_on'] = ord_enc.fit_transform(dataset[['created_on']])
    return dataset

# Normalização

## Por reescala

In [17]:
def normalize_max_min(dataset, cols):
    for feature in cols:
        max_value = dataset[feature].max()
        min_value = dataset[feature].min()
        dataset[feature] = (dataset[feature] - min_value) / (max_value - min_value)
    return dataset

## Por padronização

In [18]:
def standardizate(dataset, cols):
    for feature in cols:
        dataset[feature] = (dataset[feature] - dataset[feature].mean()) / (dataset[feature].std())
    return dataset

# Aplicando o Pré-Processamento

In [19]:
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))
df = filter_currency(df)
df = filter_latlon(df)

df = manual_selection(df)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))
df.dtypes

#Exemplos: 1000000
#Atributos: 24
#Exemplos: 495836
#Atributos: 9


created_on                object
property_type             object
lat                      float64
lon                      float64
surface_total_in_m2      float64
surface_covered_in_m2    float64
rooms                    float64
expenses                 float64
price                    float64
dtype: object

In [20]:
df.isnull().sum()

created_on                    0
property_type                 0
lat                           0
lon                           0
surface_total_in_m2      346725
surface_covered_in_m2     91855
rooms                    184491
expenses                 315895
price                         0
dtype: int64

In [21]:
df = convert_property_type(df)
df = convert_created_on(df)
df.head()

   created_on        lat        lon  surface_total_in_m2  surface_covered_in_m2  rooms  expenses   PH  apartment  house  store     price
0  2017-04-28 -23.616325 -46.676620                  NaN                   80.0    NaN    1065.0  0.0        1.0    0.0    0.0  900000.0
1  2017-05-25 -30.051750 -51.182068                 91.0                   91.0    NaN       NaN  0.0        1.0    0.0    0.0  750000.0
2  2016-02-22 -25.413214 -49.271294                  NaN                  168.0    NaN       NaN  0.0        1.0    0.0    0.0  880000.0
3  2016-09-09 -23.562290 -46.521248                  NaN                  145.0    2.0       NaN  0.0        0.0    1.0    0.0  580000.0
4  2016-09-18 -21.809600 -46.544624                138.0                  260.0    NaN       NaN  0.0        0.0    1.0    0.0  420000.0         created_on        lat        lon  surface_total_in_m2  surface_covered_in_m2  rooms  expenses   PH  apartment  house  store      price
495831  2016-05-11 -23.023577 -43.4

,created_on,lat,lon,surface_total_in_m2,surface_covered_in_m2,rooms,expenses,PH,apartment,house,store,price
0,981.0,-23.616325,-46.676620,NaN,80.0,NaN,1065.0,0.0,1.0,0.0,0.0,900000.0
1,1008.0,-30.051750,-51.182068,91.0,91.0,NaN,NaN,0.0,1.0,0.0,0.0,750000.0
2,568.0,-25.413214,-49.271294,NaN,168.0,NaN,NaN,0.0,1.0,0.0,0.0,880000.0
3,755.0,-23.562290,-46.521248,NaN,145.0,2.0,NaN,0.0,0.0,1.0,0.0,580000.0
4,764.0,-21.809600,-46.544624,138.0,260.0,NaN,NaN,0.0,0.0,1.0,0.0,420000.0


In [22]:
df.tail()

,created_on,lat,lon,surface_total_in_m2,surface_covered_in_m2,rooms,expenses,PH,apartment,house,store,price
495831,637.0,-23.023577,-43.471703,NaN,75.0,NaN,680.0,0.0,1.0,0.0,0.0,530000.0
495832,666.0,-23.577942,-46.662556,NaN,315.0,3.0,NaN,0.0,0.0,1.0,0.0,1800000.0
495833,1107.0,-30.022842,-51.193993,151.0,102.0,NaN,NaN,0.0,1.0,0.0,0.0,798000.0
495834,340.0,-22.883080,-42.027451,NaN,NaN,2.0,NaN,0.0,0.0,1.0,0.0,250000.0
495835,1088.0,-12.941992,-38.403465,NaN,112.0,1.0,941.0,0.0,1.0,0.0,0.0,760000.0


In [23]:
df = fill_expenses(df)
df = fill_surface_total_with_surface_covered(df)
df = fill_surface_covered_with_surface_total(df)
df = drop_remaining_objects_without_surfaces(df)
df.isnull().sum()*100/df.shape[0]

created_on                0.000000
lat                       0.000000
lon                       0.000000
surface_total_in_m2       0.000000
surface_covered_in_m2     0.000000
rooms                    36.198835
expenses                  0.000000
PH                        0.000000
apartment                 0.000000
house                     0.000000
store                     0.000000
price                     0.000000
dtype: float64

In [24]:
df = fill_rooms(df)
df.isnull().sum()*100/df.shape[0]

created_on               0.0
lat                      0.0
lon                      0.0
surface_total_in_m2      0.0
surface_covered_in_m2    0.0
rooms                    0.0
expenses                 0.0
PH                       0.0
apartment                0.0
house                    0.0
store                    0.0
price                    0.0
dtype: float64

In [25]:
cols = ['rooms']
df = remove_outliers(df, cols)
print("#Exemplos: {}".format(df.shape[0]))

#Exemplos: 449633


In [26]:
print(df.dtypes)
df.head()

created_on               float64
lat                      float64
lon                      float64
surface_total_in_m2      float64
surface_covered_in_m2    float64
rooms                    float64
expenses                 float64
PH                       float64
apartment                float64
house                    float64
store                    float64
price                    float64
dtype: object


,created_on,lat,lon,surface_total_in_m2,surface_covered_in_m2,rooms,expenses,PH,apartment,house,store,price
0,981.0,-23.616325,-46.676620,80.0,80.0,2.0,1065.0,0.0,1.0,0.0,0.0,900000.0
1,1008.0,-30.051750,-51.182068,91.0,91.0,2.0,0.0,0.0,1.0,0.0,0.0,750000.0
2,568.0,-25.413214,-49.271294,168.0,168.0,2.0,0.0,0.0,1.0,0.0,0.0,880000.0
3,755.0,-23.562290,-46.521248,145.0,145.0,2.0,0.0,0.0,0.0,1.0,0.0,580000.0
4,764.0,-21.809600,-46.544624,138.0,260.0,3.0,0.0,0.0,0.0,1.0,0.0,420000.0


In [27]:
df.dtypes
df.to_csv(dataset_dir + "/preprocessed.csv", index=False)

## E se tivesse normalizado

In [28]:
cols = ['created_on', 'lat', 'lon', 'rooms']
df = normalize_max_min(df, cols)

cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'expenses']
df = standardizate(df, cols)
df.head()

,created_on,lat,lon,surface_total_in_m2,surface_covered_in_m2,rooms,expenses,PH,apartment,house,store,price
0,0.786688,0.311355,0.664401,-0.001542,-0.001534,0.2,0.005237,0.0,1.0,0.0,0.0,900000.0
1,0.808340,0.124457,0.537113,-0.001539,-0.001531,0.2,-0.026410,0.0,1.0,0.0,0.0,750000.0
2,0.455493,0.259170,0.591096,-0.001515,-0.001507,0.2,-0.026410,0.0,1.0,0.0,0.0,880000.0
3,0.605453,0.312925,0.668791,-0.001522,-0.001514,0.2,-0.026410,0.0,0.0,1.0,0.0,580000.0
4,0.612670,0.363826,0.668130,-0.001524,-0.001478,0.4,-0.026410,0.0,0.0,1.0,0.0,420000.0
